## Ecommerce Product Return Data Analysis

In [64]:
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import seaborn as sns
import matplotlib.pyplot as plt
from math import sin, cos, sqrt, atan2, radians

In [45]:
# Load the dataset
df = pd.read_csv("Ecommerece_return_datadet.csv", encoding="Latin")

In [46]:
df.head(2)

,user_id,age,gender,city,traffic_source,order_id,status,product_id,product_category,product_name,product_retail_price,dc_name,d_lat,d_long,u_lat,u_long,dc2c_distance,created_at,shipped_at,delivered_at,prep_time,delivery_time,total_time
0,2480,22,F,Johns Creek,Search,3039,Complete,13998,Accessories,Angelina Double-layer / Microfiber lined Cozy ...,14.99,Mobile AL,30.6944,-88.0431,34.026725,-84.149576,520.518004,2024-01-19 05:03:01.000000 UTC,2024-01-21 17:06:00.000000 UTC,2024-01-22 14:59:00.000000 UTC,3602,1313,4915
1,2644,61,F,McKinney,Search,3226,Complete,13998,Accessories,Angelina Double-layer / Microfiber lined Cozy ...,14.99,Mobile AL,30.6944,-88.0431,33.159099,-96.694238,860.877231,2023-12-30 21:43:54.000000 UTC,2023-12-29 15:29:00.000000 UTC,2024-01-01 21:57:00.000000 UTC,-1814,4708,2894


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   user_id               3333 non-null   int64  
 1   age                   3333 non-null   int64  
 2   gender                3333 non-null   object 
 3   city                  3311 non-null   object 
 4   traffic_source        3333 non-null   object 
 5   order_id              3333 non-null   int64  
 6   status                3333 non-null   object 
 7   product_id            3333 non-null   int64  
 8   product_category      3333 non-null   object 
 9   product_name          3333 non-null   object 
 10  product_retail_price  3333 non-null   float64
 11  dc_name               3333 non-null   object 
 12  d_lat                 3333 non-null   float64
 13  d_long                3333 non-null   float64
 14  u_lat                 3333 non-null   float64
 15  u_long               

In [48]:
## Interesting questions
# 1. Which age groups are particularly returing the maximum product
# 2. Do gender affect the product status
# 3. Which city saw the maximum return of product
# 4. The maximum product return is from which traffic source
# 5. Which product category saw maxmium return of product.
# 6. Do total time affects the product return status.
# 7. 

#### 1. Handle the Missing Value

In [49]:
## Handle the missing values
(df.isnull().sum()/len(df))*100
## 0.66% of data is missing in city column

user_id                 0.000000
age                     0.000000
gender                  0.000000
city                    0.660066
traffic_source          0.000000
order_id                0.000000
status                  0.000000
product_id              0.000000
product_category        0.000000
product_name            0.000000
product_retail_price    0.000000
dc_name                 0.000000
d_lat                   0.000000
d_long                  0.000000
u_lat                   0.000000
u_long                  0.000000
dc2c_distance           0.000000
created_at              0.000000
shipped_at              0.000000
delivered_at            0.000000
prep_time               0.000000
delivery_time           0.000000
total_time              0.000000
dtype: float64

In [50]:
## Let's replace the missing value with the mode i.e Newyork
df["city"].value_counts()
df["city"] = df["city"].fillna("New York")

#### 2. Check Duplicate Values

In [51]:
# find duplicate rows
duplicate_rows = df.duplicated().sum()

# print duplicate rows
print(duplicate_rows)
## No duplicate rows

0


#### 3. Data Accuracy

In [52]:
df.describe()
## There seems to have same accuracy with timing columns, they can not be negative

,user_id,age,order_id,product_id,product_retail_price,d_lat,d_long,u_lat,u_long,dc2c_distance,prep_time,delivery_time,total_time
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,50460.922892,41.207921,63335.648665,15250.132913,59.888251,34.864454,-88.800364,36.721204,-94.827461,1744.417627,761.961896,3608.727873,4370.689769
std,28232.558956,17.008603,35422.780740,8395.555830,68.964125,4.420372,11.766403,5.246989,17.142256,1128.675819,2426.140776,2102.143854,3228.529915
min,28.000000,12.000000,44.000000,23.000000,0.490000,29.760400,-118.250000,19.716157,-157.879335,4.765293,-5691.000000,2.000000,-4972.000000
25%,26728.000000,27.000000,33559.000000,7856.000000,24.000000,30.694400,-90.066700,33.435376,-112.066741,900.523434,-766.000000,1762.000000,2224.000000
50%,51501.000000,41.000000,64611.000000,16033.000000,39.990002,34.050000,-88.043100,36.784596,-88.921171,1501.281104,1143.000000,3684.000000,4473.000000
75%,74778.000000,56.000000,93886.000000,22428.000000,69.989998,39.950000,-79.933300,40.667261,-80.675970,2558.741570,2677.000000,5418.000000,6800.000000
max,99844.000000,70.000000,125281.000000,29119.000000,999.000000,41.836900,-73.783400,61.493794,-69.564192,7997.316055,4544.000000,7195.000000,11570.000000


In [53]:
df.head()

,user_id,age,gender,city,traffic_source,order_id,status,product_id,product_category,product_name,product_retail_price,dc_name,d_lat,d_long,u_lat,u_long,dc2c_distance,created_at,shipped_at,delivered_at,prep_time,delivery_time,total_time
0,2480,22,F,Johns Creek,Search,3039,Complete,13998,Accessories,Angelina Double-layer / Microfiber lined Cozy ...,14.99,Mobile AL,30.6944,-88.0431,34.026725,-84.149576,520.518004,2024-01-19 05:03:01.000000 UTC,2024-01-21 17:06:00.000000 UTC,2024-01-22 14:59:00.000000 UTC,3602,1313,4915
1,2644,61,F,McKinney,Search,3226,Complete,13998,Accessories,Angelina Double-layer / Microfiber lined Cozy ...,14.99,Mobile AL,30.6944,-88.0431,33.159099,-96.694238,860.877231,2023-12-30 21:43:54.000000 UTC,2023-12-29 15:29:00.000000 UTC,2024-01-01 21:57:00.000000 UTC,-1814,4708,2894
2,95843,38,F,Fruit Cove,Search,120285,Returned,14180,Accessories,Kipling New Money Deluxe Wallet,34.00,Mobile AL,30.6944,-88.0431,30.072132,-81.603143,621.517846,2023-07-18 03:32:21.000000 UTC,2023-07-20 15:54:00.000000 UTC,2023-07-23 19:52:00.000000 UTC,3621,4558,8179
3,49815,57,M,Columbus,Search,62459,Returned,29032,Accessories,Leather in Chicago Mens Trifold Genuine Leathe...,7.99,Mobile AL,30.6944,-88.0431,39.955191,-82.930252,1128.859971,2023-09-02 21:56:57.000000 UTC,2023-09-01 12:47:00.000000 UTC,2023-09-04 12:49:00.000000 UTC,-1989,4322,2333
4,38211,41,F,Parker,Search,48054,Returned,14063,Accessories,LibbySue-Classic Cashmere Feel Winter Scarf in...,20.00,Mobile AL,30.6944,-88.0431,39.482554,-104.770184,1804.019236,2023-05-28 09:18:02.000000 UTC,2023-05-25 11:50:00.000000 UTC,2023-05-27 05:30:00.000000 UTC,-4168,2500,-1668


In [55]:
df[df["order_id"] == 103842]

,user_id,age,gender,city,traffic_source,order_id,status,product_id,product_category,product_name,product_retail_price,dc_name,d_lat,d_long,u_lat,u_long,dc2c_distance,created_at,shipped_at,delivered_at,prep_time,delivery_time,total_time
795,82771,37,M,Phoenix,Facebook,103842,Complete,28584,Accessories,Guide Gear Leather Rabbit Fur Hat,39.990002,Houston TX,29.7604,-95.3698,33.430418,-112.198568,1643.207542,2023-12-12 02:13:21.000000 UTC,2023-12-11 14:58:00.000000 UTC,2023-12-15 02:40:00.000000 UTC,-675,5022,4347
2350,82771,37,M,Phoenix,Facebook,103842,Complete,22241,Pants,Savane Men's Flat Front No-Iron Twill Pant,39.990002,Los Angeles CA,34.0500,-118.2500,33.430418,-112.198568,563.689610,2023-12-12 03:30:44.000000 UTC,2023-12-11 14:58:00.000000 UTC,2023-12-15 02:40:00.000000 UTC,-752,5022,4270
2509,82771,37,M,Phoenix,Facebook,103842,Complete,16769,Tops & Tees,Skulls Guns and Roses Western Shirt Gunslinger...,39.990002,Los Angeles CA,34.0500,-118.2500,33.430418,-112.198568,563.689610,2023-12-13 02:23:51.000000 UTC,2023-12-11 14:58:00.000000 UTC,2023-12-15 02:40:00.000000 UTC,-2125,5022,2897
2948,82771,37,M,Phoenix,Facebook,103842,Complete,27204,Sleep & Lounge,Harbor Bay Big & Tall French Terry Lounge Pants,64.000000,Philadelphia PA,39.9500,-75.1667,33.430418,-112.198568,3355.347333,2023-12-11 01:23:27.000000 UTC,2023-12-11 14:58:00.000000 UTC,2023-12-15 02:40:00.000000 UTC,814,5022,5836


In [61]:
# We can observe that there can be multiple product within a single orderid
# Their created time is different but product shipped and delivered time is same
#Distance calculate 180format
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in kilometers
    # Convert to radians
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = sin(dlat/2)**2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c
    return distance

In [62]:
##create endpoint distribution center (name) and end to customer distance (e2c_dist)
def find_closest_dc(row):
    cust_lat = row["u_lat"]  ## user latitude
    cust_lon = row["u_long"]

    dist = []
    for index, dc_row in df.iterrows():
        dc_lat = dc_row['d_lat']
        dc_lon = dc_row['d_long']

        distance = calculate_distance(cust_lat, cust_lon, dc_lat, dc_lon)
        dist.append(distance)

    closest_dc_index = min(range(len(dist)), key=dist.__getitem__)
    closest_dc = df_dc.at[closest_dc_index, 'dc_name']
    e_lat = df_dc.at[closest_dc_index, 'd_lat']
    e_long = df_dc.at[closest_dc_index, 'd_long']
    distance = dist[closest_dc_index]

    return pd.Series([closest_dc, distance,e_lat,e_long]
                     , index=['end_dc', 'e2c_dist','e_lat','e_long'])


In [65]:
#calculate start to end distance (s2e_dist)
def calculate_s2e_distance(row):
    e_lat = row['e_lat']
    e_long = row['e_long']
    d_lat = row['d_lat']
    d_long = row['d_long']
    s2e_distance = calculate_distance(e_lat, e_long, d_lat, d_long)
    return s2e_distance

#combine Fn
def distance_calculate(df):
    df[['end_dc', 'e2c_dist', 'e_lat', 'e_long']] = df.apply(find_closest_dc, axis=1)
    df['s2e_dist'] = df.apply(calculate_s2e_distance, axis=1)
    return df

#Apply
df = distance_calculate(df)

NameError: name 'df_dc' is not defined